## Useful functions

밑에있는 함수들을 사용하렴.. print 대신 print_log를 사용하면 중간에 이 노트북을 꺼도 된단다

In [1]:
def print_log(*args, **kwargs) :
    with open('running_log.log', 'a') as f :
        print(*args, file=f, **kwargs)
    print(*args, **kwargs)

아직 정해진게 아무것도 없다...

### some ideas
1) encoding
- image의 형태는 변하지 않고 위치만 변한다
- 따라서 base image를 추출하면 편함
- Red number는 항상 위에 있으므로 추출 편함
- Green number는 가장 green component가 많은 image에서 추출
- Red number는 좌표를 얻기 편하다
- Green number는 가장 일치하는 점이 많은 곳에서 좌표
- Green number가 어렵다 흠..

2) RNN
- 좌표의 sequence 처리

3) decoding
- 좌표 + base image

## 구현 기준

숫자는 24 by 24 pixel

중심을 기준으로 좌표 설정. 중심은 0~23, 0~23중에서 index를 11로 설정

# Step 0. Import

In [2]:
import os
import numpy as np
import tensorflow as tf
from pathlib import Path
import random 
import time

import tensorflow as tf
from tensorflow.contrib import rnn, cudnn_rnn
from tensorflow.keras import layers

이거 만드는거 개꿀잼

# ㄷㄷ 에반디

### ㅋㅋㅋㅋ댓글다는거 ㄹㅇ 핵웃기넼ㅋㅋㅋㅋㅋㅋㅋㅋㅋ

# Step 2. RNN

## Configuration of Neural Network

In [3]:
# Training Parameters
learning_rate = 0.001
training_steps = 10
batch_size = 128
display_step = 200

# Network Parameters
input_channel = 4 # 2 for coordinates of red number, 2 for coordinates of green number
num_input = 10
output_channel = 4 # same as input
num_output = 10 # have to predict 10 future frames
timesteps = 19 # timesteps (the number of rows)
num_hidden = 4 # hidden layer num of features
num_layers = 1 # the number of LSTM's layers

# Loading data


data shape은 원하는대로 바꾸거라

지금은 red, green 따로해서 input output 있음

In [4]:
data_set_size = 10000
red_train_input = np.zeros([data_set_size, 10, 2], np.uint8)  # data set 크기, sequence 안에서 input은 10개, x좌표 y좌표 해서 2개
red_train_output = np.zeros([data_set_size, 10, 2], np.uint8) #위와 마찬가지
green_train_input = np.zeros([data_set_size, 10, 2], np.uint8)
green_train_output = np.zeros([data_set_size, 10, 2], np.uint8)

red_num_coord_path = "./processed_data/red_num_cordinates/"
green_num_coord_path = "./processed_data/green_num_cordinates/"
for i in range(data_set_size) :
    red_seq_path = os.path.join(red_num_coord_path, '%04d' % i) 
    green_seq_path = os.path.join(green_num_coord_path, '%04d' % i)
    
    with open(red_seq_path) as f :
        for idx in range(10) :
            sp_list = f.readline().split()
            red_train_input[i][idx][0] = int(sp_list[0])
            red_train_input[i][idx][1] = int(sp_list[1])
        for idx in range(10) :
            sp_list = f.readline().split()
            red_train_output[i][idx][0] = int(sp_list[0])
            red_train_output[i][idx][1] = int(sp_list[1])
            
    with open(green_seq_path) as f :
        for idx in range(10) :
            sp_list = f.readline().split()
            green_train_input[i][idx][0] = int(sp_list[0])
            green_train_input[i][idx][1] = int(sp_list[1])
        for idx in range(10) :
            sp_list = f.readline().split()
            green_train_output[i][idx][0] = int(sp_list[0])
            green_train_output[i][idx][1] = int(sp_list[1])

            
# Examples
# print(red_train_input[0])
# print(red_train_output[0])
# print(green_train_input[2031])
# print(green_train_output[2031])

# Train these data!

In [5]:
# Data onehot encoding
tmp_i = red_train_input[:, :, 0].copy() - 11
tmp_o = red_train_output[:, :, 0].copy() - 11

b = np.zeros((10000, 10, 41))
for i in range(10000) :
    b[i, np.arange(10), tmp_i[i]] = 1
input_batch = b

b = np.zeros((10000, 10, 41))
for i in range(10000) :
    b[i, np.arange(10), tmp_i[i]] = 1
output_batch = b

target_batch = tmp_o


In [11]:
#########
# 신경망 모델 구성
######
# Seq2Seq 모델은 인코더의 입력과 디코더의 입력의 형식이 같다.
# [batch size, time steps, input size]

n_input = 41
n_hidden = 128
n_class = n_input

enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
# [batch size, time steps]
targets = tf.placeholder(tf.int64, [None, None])


# 인코더 셀을 구성한다.
with tf.variable_scope('encode', reuse = tf.AUTO_REUSE):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)

    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype=tf.float32)

# 디코더 셀을 구성한다.
with tf.variable_scope('decode', reuse = tf.AUTO_REUSE):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)

    # Seq2Seq 모델은 인코더 셀의 최종 상태값을
    # 디코더 셀의 초기 상태값으로 넣어주는 것이 핵심.
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, initial_state=enc_states, dtype=tf.float32)

model = tf.layers.dense(outputs, n_class, activation=None)

targets_float = tf.cast(targets, tf.float32)


cost = tf.reduce_mean((tf.square(tf.cast(tf.argmax(model) - targets, tf.float32))))
            #tf.nn.(
            #    logits=model, labels=targets))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

ValueError: No gradients provided for any variable, check your graph for ops that do not support gradients, between variables ["<tf.Variable 'encode/rnn/basic_rnn_cell/kernel:0' shape=(169, 128) dtype=float32>", "<tf.Variable 'encode/rnn/basic_rnn_cell/bias:0' shape=(128,) dtype=float32>", "<tf.Variable 'decode/rnn/basic_rnn_cell/kernel:0' shape=(169, 128) dtype=float32>", "<tf.Variable 'decode/rnn/basic_rnn_cell/bias:0' shape=(128,) dtype=float32>", "<tf.Variable 'dense/kernel:0' shape=(128, 41) dtype=float32_ref>", "<tf.Variable 'dense/bias:0' shape=(41,) dtype=float32_ref>", "<tf.Variable 'dense_1/kernel:0' shape=(128, 41) dtype=float32_ref>", "<tf.Variable 'dense_1/bias:0' shape=(41,) dtype=float32_ref>", "<tf.Variable 'dense_2/kernel:0' shape=(128, 41) dtype=float32_ref>", "<tf.Variable 'dense_2/bias:0' shape=(41,) dtype=float32_ref>", "<tf.Variable 'dense_3/kernel:0' shape=(128, 41) dtype=float32_ref>", "<tf.Variable 'dense_3/bias:0' shape=(41,) dtype=float32_ref>"] and loss Tens

In [8]:
#########
# 신경망 모델 학습
######
total_epoch = 10000

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={enc_input: input_batch,
                                  dec_input: output_batch,
                                  targets: target_batch})

    print('Epoch:', '%04d' % (epoch + 1),
          'cost =', '{:.6f}'.format(loss))

print('최적화 완료!')

prediction = tf.argmax(model, 2)

InvalidArgumentError: Incompatible shapes: [10000,10,41] vs. [10000,10]
	 [[node sub (defined at <ipython-input-6-3483c43ad79d>:38) ]]
	 [[node Mean (defined at <ipython-input-6-3483c43ad79d>:38) ]]

Caused by op 'sub', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/snulife/Documents/KJH/ML/ML_3.6/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/snulife/Documents/KJH/ML/ML_3.6/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/snulife/Documents/KJH/ML/ML_3.6/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/snulife/Documents/KJH/ML/ML_3.6/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 148, in start
    self.asynci

In [ ]:
print(target_batch[:100])
print(sess.run(
    prediction, 
    feed_dict = {
        enc_input: input_batch[:100], 
        dec_input: output_batch[:100],
        targets: target_batch[:100]
    }
))